In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
train = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv"))

In [3]:
test = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv"))

In [4]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
train['Region_Code'] = train['Region_Code'].astype('int16')
train['Policy_Sales_Channel'] = train['Policy_Sales_Channel'].astype('int16')

In [60]:
df_1 = train[train['Response'] == 1].sample(frac=0.4, random_state=42)
df_0 = train[train['Response'] == 0].sample(frac=0.08, random_state=42)
df = pd.concat([df_1, df_0], axis=0)
print(df_1.describe())
print(df_0.describe())

                 id            Age  Driving_License    Region_Code  \
count  5.660240e+05  566024.000000    566024.000000  566024.000000   
mean   5.756191e+06      43.284145         0.999104      26.849180   
std    3.317521e+06      11.488728         0.029915      11.287225   
min    1.000000e+00      20.000000         0.000000       0.000000   
25%    2.890234e+06      35.000000         1.000000      26.000000   
50%    5.751672e+06      43.000000         1.000000      28.000000   
75%    8.632095e+06      50.000000         1.000000      29.000000   
max    1.150478e+07      83.000000         1.000000      52.000000   

       Previously_Insured  Annual_Premium  Policy_Sales_Channel  \
count       566024.000000   566024.000000         566024.000000   
mean             0.002526    31872.037108             90.442771   
std              0.050200    17698.924378             55.889491   
min              0.000000     2630.000000              1.000000   
25%              0.000000    25957

In [61]:
X = df.drop(['Response', 'id'], axis=1)
y = df['Response']

In [62]:
X.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
5592646,Female,38,1,41,0,1-2 Year,Yes,36774.0,26,65
11249674,Female,36,1,17,0,1-2 Year,Yes,28571.0,124,286
6803340,Male,56,1,15,0,1-2 Year,Yes,2630.0,156,209
2136921,Female,33,1,28,0,1-2 Year,Yes,36585.0,158,164
1078493,Female,59,1,28,0,1-2 Year,Yes,48945.0,124,161


In [63]:
cat_features = [0, 2, 3, 4, 5, 6, 8]

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=22)

In [77]:
import gc
# 가비지 컬렉션 수행 전, 객체 수 확인
print("Before GC:", len(gc.get_objects()))

# 가비지 컬렉션 수행
gc.collect()

# 가비지 컬렉션 수행 후, 객체 수 확인
print("After GC:", len(gc.get_objects()))

Before GC: 758425
After GC: 725451


In [80]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=50,
    learning_rate=0.4,
    depth=8,
    one_hot_max_size=3,
    custom_loss=['AUC', 'Recall']
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
)

0:	learn: 0.5605532	test: 0.5605473	best: 0.5605473 (0)	total: 350ms	remaining: 17.2s
1:	learn: 0.5022464	test: 0.5024121	best: 0.5024121 (1)	total: 705ms	remaining: 16.9s
2:	learn: 0.4709363	test: 0.4712192	best: 0.4712192 (2)	total: 1.05s	remaining: 16.4s
3:	learn: 0.4522636	test: 0.4527112	best: 0.4527112 (3)	total: 1.34s	remaining: 15.5s
4:	learn: 0.4412591	test: 0.4417766	best: 0.4417766 (4)	total: 1.67s	remaining: 15.1s
5:	learn: 0.4332745	test: 0.4337528	best: 0.4337528 (5)	total: 2.01s	remaining: 14.8s
6:	learn: 0.4260460	test: 0.4265603	best: 0.4265603 (6)	total: 2.35s	remaining: 14.5s
7:	learn: 0.4222300	test: 0.4227776	best: 0.4227776 (7)	total: 2.65s	remaining: 13.9s
8:	learn: 0.4200410	test: 0.4207160	best: 0.4207160 (8)	total: 2.96s	remaining: 13.5s
9:	learn: 0.4177318	test: 0.4183446	best: 0.4183446 (9)	total: 3.28s	remaining: 13.1s
10:	learn: 0.4159664	test: 0.4165016	best: 0.4165016 (10)	total: 3.58s	remaining: 12.7s
11:	learn: 0.4148758	test: 0.4154245	best: 0.4154245

In [81]:
y_pred = clf.predict(data=X_val)

In [82]:
def calc_accuracy(y_pred, y_val):
    acc = y_val == y_pred
    i = 0
    for d in acc:
        if d == True:
            i+=1
    s, n = 0, 0
    for j in y_pred:
        n+=1
        if j==1:
            s += 1
    print(i/acc.size, s, n)

In [83]:
calc_accuracy(y_pred, y_val)

0.7922750344084298 70946 137321


In [84]:
X_all = train.drop(['Response', 'id'], axis=1)
y_pred_all = clf.predict(data=X_all)

In [85]:
#please execute once
y_all = train['Response']

In [86]:
calc_accuracy(y_pred_all, y_all)

0.7505780631698183 3938888 11504798
